# MODEL 2 PERSISTENCE

In [ ]:
import math
import phat
import itertools
import sys
import numpy as np
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import persim
from persim import images_weights

In [ ]:
modname=1
funname=1

extremal_values=np.loadtxt("MinimiMassimiFunzioni.txt")

In [ ]:
# COLOR OFF FILE and LOAD FILTERING FUNCTION

min_val=extremal_values[0][funname-1]
max_val=extremal_values[1][funname-1]

FUNfile = open("functions/{}-{}.usr".format(modname,funname), 'r')

Fv = np.loadtxt(FUNfile)
cmap = mpl.cm.coolwarm
norm = mpl.colors.SymLogNorm(linthresh=1, vmin=min_val, vmax=max_val)
Cv = cmap(norm(Fv))

FUNfile.close()

OFFfile = open("data/{}.off".format(modname), 'r')

COFFfile = open("data_with_functions/{}-{}.off".format(modname,funname), "w")


for line in itertools.islice(OFFfile, 0, 2):  # start=17, stop=None
    pass


COFFfile.write("COFF\n")
COFFfile.write("\n")

line = OFFfile.readline()

COFFfile.write(line)

nv, nt = (int(line.split()[0]), int(line.split()[1]))

i=0

for line in itertools.islice(OFFfile, 0, nv):
    color=Cv[i]
    newline=line.rstrip("\n") + " {} {} {} {}\n".format(color[0], color[1], color[2], color[3])
    COFFfile.write(newline)
    i=i+1

for line in itertools.islice(OFFfile, 0, nt):
    COFFfile.write(line)

    
OFFfile.close()
COFFfile.close()

In [ ]:
# LOAD SURFACE

INfile = open("data/{}.off".format(modname), "r")

for line in itertools.islice(INfile, 0, 2):  # start=17, stop=None
    pass


line = INfile.readline()
nv, nt = int(line.split()[0]), int(line.split()[1])

vertices = []
edges = []
triangles = []

# LOADING VERTICES
for i in range(nv):
    vertices.append([i])

# LOADING TRIANGLES and EDGES
E = set()
for line in itertools.islice(INfile, nv, None):

    a = int(line.split()[1])
    b = int(line.split()[2])
    c = int(line.split()[3])

    triangles.append([a, b, c])

    edges = [[a, b], [a, c], [b, c]]
    for edge in edges:
        edge.sort()
        E.add(tuple(edge))

edges = list(list(s) for s in E)

print("\nLoaded Model {}".format(modname))
print("# Vertices:{}".format(len(vertices)))
print("# Edges:{}".format(len(edges)))
print("# Triangles:{}".format(len(triangles)))

INfile.close()

In [ ]:
# CREATE BOUNDARY MATRIX


# function extended to all the simplices

def F(s, Fv):
    return Fv[s].astype(np.float64).max()


# storing simplex indices

D = vertices+edges+triangles

D.sort(key=lambda x: (F(x, Fv), len(x)))

# Here, we adopt a map to quickly find the index of each simplex (see next steps)

t_map = {}
count = 0
for i in D:
    i.sort()
    t_map[tuple(i)] = count
    count += 1
    
    
# boundary of a simplex

def Comb(L):
    LL = []
    for i in range(len(L)):
        LLL = []
        for j in range(len(L)):
            if i != j:
                LLL.append(L[j])
        LLL.sort()
        LL.append(LLL)
    return LL


def Bound(el, S):
    Indexes = []
    for s in Comb(el):
        if s:
            Indexes.append(S[tuple(s)])
    return sorted(Indexes)


# boundary matrix
    
C = [() for x in range(len(D))]

for i in range(0, len(D)):
    C[i] = (len(D[i])-1, Bound(D[i], t_map))
    
boundary_matrix = phat.boundary_matrix(representation=phat.representations.vector_vector)

boundary_matrix.columns = C

In [ ]:
# COMPUTE PERSISTENT HOMOLOGY

pairs = boundary_matrix.compute_persistence_pairs()

In [ ]:
# COMPUTE real PERSISTENT HOMOLOGY


sF = np.zeros(len(D))
leng = np.zeros(len(D))
for i in range(len(D)):
    sF[i] = F(D[i], Fv)
    leng[i] = len(D[i])

# Convert pairs to np.array
pairs = np.array(pairs)
pairs_flatten = pairs.flatten()

# Initialize the Indices corresponding to all the simplexes
indices = np.arange(len(D))

# Assign a negative value the pairs found
indices[pairs_flatten] = -1

# Filter out the indices corresponding to a pair
indices_filt = indices[indices > -1]

birth_minus_death = sF[pairs[:, 0]] - sF[pairs[:, 1]]

nontrivial_bd = birth_minus_death[birth_minus_death < 0]

# finite pairs:
num_finite_pairs = nontrivial_bd.shape[0]

# infinite pairs
num_infinite_pairs = indices_filt.shape[0]

# real_pairs track the pairs with non-trivial life (death - birth > 0)
rescaledPairs = np.zeros(shape=(num_finite_pairs + num_infinite_pairs, 2))
degree = np.zeros(num_finite_pairs + num_infinite_pairs)

# rescale non trivial (finite) pairs
rescaledPairs[:num_finite_pairs, 0] = sF[pairs[birth_minus_death < 0, 0]]
rescaledPairs[:num_finite_pairs, 1] = sF[pairs[birth_minus_death < 0, 1]]
degree[:num_finite_pairs] = leng[pairs[birth_minus_death < 0, 0]]-1

# rescale non trivial (infinite) pairs
rescaledPairs[num_finite_pairs:, 0] = sF[indices_filt]
rescaledPairs[num_finite_pairs:, 1] = np.inf
# rescaledPairs[num_finite_pairs:, 1] = max_val + 1
degree[num_finite_pairs:] = leng[indices_filt] - 1

# Here, the REAL persistence diagrams
dZero = rescaledPairs[degree == 0, :]
dOne = rescaledPairs[degree == 1, :]
dTwo = rescaledPairs[degree == 2, :]

In [ ]:
len(pairs)

In [ ]:
len(dZero)+len(dOne)+len(dTwo)

In [ ]:
# VISUALIZE PERSISTENCE DIAGRAMS

persim.plot_diagrams([dZero, dOne, dTwo], title="PD of Model {} w.r.t. Function {}".format(modname, funname))

# plt.savefig("images/Diagram_{}_{}.png".format(modname, funname))

plt.show()